In [1]:
import math
import numpy as np
import pandas as pd
from geopy import distance
from typing import Tuple

df = catalog.load('enriched_df')

[03/23/23 23:55:06] INFO     Loading data from 'enriched_df' (CSVDataSet)...                    ]8;id=680774;file:///home/cdanmontoya/.cache/pypoetry/virtualenvs/gpx-dashboard-N2OiNTXu-py3.9/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=614718;file:///home/cdanmontoya/.cache/pypoetry/virtualenvs/gpx-dashboard-N2OiNTXu-py3.9/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [3]:
df.groupby('city').count()

,trip,step,time,latitude,longitude,elevation,neighbourhood,suburb,postcode
city,,,,,,,,,
Medellín,34908,34908,34908,34908,34908,34908,34908,34908,34908


In [39]:
def get_partial_distance(actual_position: Tuple[float, float, float], previous_position: Tuple[float, float, float]) -> float:   
    if math.isnan(previous_position[0]):
        return 0
    
    flat_distance = distance.distance(previous_position[:2], actual_position[:2]).km
    euclidian_distance = math.sqrt(flat_distance**2 + (actual_position[2] - previous_position[2])**2)
    
    return euclidian_distance
    

In [46]:
def get_distance(df: pd.DataFrame) -> pd.DataFrame:
    df[['previous_latitude','previous_longitude', 'previous_elevation']] = df.groupby('trip')[['latitude','longitude', 'elevation']].shift()
    
    # TODO: Vectorize this call to improve performance
    df['partial_distance'] = df.apply(lambda x: get_partial_distance((x['latitude'], x['longitude'], x['elevation']), (x['previous_latitude'], x['previous_longitude'], x['previous_elevation'])), axis=1)
    
    return df.groupby('trip')['partial_distance'].sum().to_frame().reset_index().rename({'partial_distance': 'distance'})
    

In [47]:
jeje = get_distance(df)

In [48]:
jeje

,trip,partial_distance
0,data/01_raw/03-Mar-2023-1714.gpx,252.933885
1,data/01_raw/18-Mar-2023-1213.gpx,46.926380
2,data/01_raw/18-Mar-2023-1809Scy.gpx,102.004968
3,data/01_raw/18-Mar-2023-1826.gpx,551.971400
4,data/01_raw/19-Mar-2023-1822.gpx,5348.098908
5,data/01_raw/19-mar.-2023-1712.gpx,503.265681
6,"data/01_raw/2023-03-18_Mar_18,_2023_5_54_26_PM...",422.009209
7,"data/01_raw/2023-03-19_Mar_19,_2023_3_17_31_PM...",3847.512123
8,data/01_raw/recovery.01-Mar-2022-1533.gpx,0.974889
9,data/01_raw/recovery.05-Mar-2022.1025.gpx,23913.808764


In [45]:
jeje.groupby('trip')['partial_distance'].sum().to_frame().reset_index()

,trip,partial_distance
0,data/01_raw/03-Mar-2023-1714.gpx,252.933885
1,data/01_raw/18-Mar-2023-1213.gpx,46.926380
2,data/01_raw/18-Mar-2023-1809Scy.gpx,102.004968
3,data/01_raw/18-Mar-2023-1826.gpx,551.971400
4,data/01_raw/19-Mar-2023-1822.gpx,5348.098908
5,data/01_raw/19-mar.-2023-1712.gpx,503.265681
6,"data/01_raw/2023-03-18_Mar_18,_2023_5_54_26_PM...",422.009209
7,"data/01_raw/2023-03-19_Mar_19,_2023_3_17_31_PM...",3847.512123
8,data/01_raw/recovery.01-Mar-2022-1533.gpx,0.974889
9,data/01_raw/recovery.05-Mar-2022.1025.gpx,23913.808764


In [66]:
df.apply(lambda x: print(type(x)), axis=1)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.S

0        None
1        None
2        None
3        None
4        None
         ... 
32749    None
32750    None
32751    None
32752    None
32753    None
Length: 32754, dtype: object

In [33]:
get_partial_distance((43.668613, 40.258916, 0.976), (43.658852, 40.250839, 1.475))

1.3599867052635133

In [ ]:
df.apply()

In [3]:
df['prev_value'] = df.groupby('trip')['time'].shift()

In [22]:
df[df.trip == 'data/01_raw/recovery.25-May-2022-0907.gpx']

,trip,step,time,latitude,longitude,elevation,neighbourhood,suburb,city,postcode,prev_value
32096,data/01_raw/recovery.25-May-2022-0907.gpx,0,2022-05-25 21:24:46.433000+00:00,6.208559,-75.571630,1558.78,Astorga,Comuna 14 - El Poblado,Medellín,50021,NaN
32097,data/01_raw/recovery.25-May-2022-0907.gpx,0,2022-05-26 14:05:32.284000+00:00,6.268901,-75.593216,1548.51,Astorga,Comuna 14 - El Poblado,Medellín,50021,2022-05-25 21:24:46.433000+00:00
32098,data/01_raw/recovery.25-May-2022-0907.gpx,1,2022-05-26 14:05:34+00:00,6.268870,-75.593087,1551.91,Astorga,Comuna 14 - El Poblado,Medellín,50021,2022-05-26 14:05:32.284000+00:00
32099,data/01_raw/recovery.25-May-2022-0907.gpx,2,2022-05-26 14:05:36+00:00,6.268792,-75.593007,1551.53,Astorga,Comuna 14 - El Poblado,Medellín,50021,2022-05-26 14:05:34+00:00
32100,data/01_raw/recovery.25-May-2022-0907.gpx,3,2022-05-26 14:05:37+00:00,6.268722,-75.592900,1551.37,Astorga,Comuna 14 - El Poblado,Medellín,50021,2022-05-26 14:05:36+00:00
...,...,...,...,...,...,...,...,...,...,...,...
32749,data/01_raw/recovery.25-May-2022-0907.gpx,652,2022-05-26 14:32:21+00:00,6.208590,-75.571338,1553.61,Astorga,Comuna 14 - El Poblado,Medellín,50021,2022-05-26 14:32:15+00:00
32750,data/01_raw/recovery.25-May-2022-0907.gpx,653,2022-05-26 14:32:27+00:00,6.208485,-75.571341,1548.82,Astorga,Comuna 14 - El Poblado,Medellín,50021,2022-05-26 14:32:21+00:00
32751,data/01_raw/recovery.25-May-2022-0907.gpx,654,2022-05-26 14:32:34+00:00,6.208457,-75.571435,1549.02,Astorga,Comuna 14 - El Poblado,Medellín,50021,2022-05-26 14:32:27+00:00
32752,data/01_raw/recovery.25-May-2022-0907.gpx,655,2022-05-26 14:32:41+00:00,6.208468,-75.571525,1559.75,Astorga,Comuna 14 - El Poblado,Medellín,50021,2022-05-26 14:32:34+00:00


In [7]:
df.trip.value_counts()

data/01_raw/recovery.05-Mar-2022.1025.gpx             18593
data/01_raw/19-Mar-2023-1822.gpx                       7142
data/01_raw/2023-03-19_Mar_19,_2023_3_17_31_PM.gpx     1693
data/01_raw/19-mar.-2023-1712.gpx                      1418
data/01_raw/18-Mar-2023-1826.gpx                       1345
data/01_raw/2023-03-18_Mar_18,_2023_5_54_26_PM.gpx     1001
data/01_raw/recovery.25-May-2022-0907.gpx               658
data/01_raw/03-Mar-2023-1714.gpx                        488
data/01_raw/18-Mar-2023-1809Scy.gpx                     358
data/01_raw/18-Mar-2023-1213.gpx                         55
data/01_raw/recovery.01-Mar-2022-1533.gpx                 3
Name: trip, dtype: int64

In [ ]:
""
""